<a href="https://colab.research.google.com/github/dirkenglund/2023.01.CQN_school/blob/main/MIWEN_exact_mixer_with_carrier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time, os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from scipy.constants import speed_of_light as c
import scipy.io
from collections import OrderedDict
import torch.fft as tfft
import datetime
import json

%matplotlib inline

# gpu = torch.device("mps")
# gpu = torch.device("cpu")

# ## Sets everything to double point precision (use with gradcheck)
# torch.set_default_dtype(torch.float64)

In [3]:
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available")

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

GPU is available: Tesla T4
Using device: cuda


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mutual_info_score

In [4]:
speedoflight = 3e8
epsilon0 = 8.85e-12

kb = 1.38e-23; T = 300
hbar = 1.05e-34

eleccharge = 1.6e-19

In [5]:
def boseeinstein(omega, T):

    return 1/(torch.exp(hbar*omega/(kb*T))-1)

In [5]:
# A list of transforms to perform on all image data
# First turn image data into pytorch tensor, then normalize it with mean and std of 0.5
# (only one channel per image) since there's only one element in each tuple

# img_size = (14, 14)
img_size = (7, 7)
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(img_size, interpolation=2), torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,))])

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,), (0.5,))])

# Downloading the training MNIST data and applying the above transform
# (The data type is a dataset, iterated by 60000 tuples, where the first element
# of each tuple contains the tensor with the data, and the second element is an integer)
# Note: each image in both the trainset and testset have shape [1, 28, 28]
dataset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

num_training = 50000
num_validation = 10000 # 10000
random_numbers = np.random.choice(np.arange(60000), size=num_training+num_validation, replace=False)

# Convert the NumPy range to a Python list
subset_indices = list(random_numbers)

# Create the subset using torch.utils.data.Subset
subset_dataset = torch.utils.data.Subset(dataset, subset_indices)

trainset, validset = torch.utils.data.random_split(subset_dataset, [num_training, num_validation])

# Downloading the test MNIST data and applying the above transform
# (The data type is a dataset, iterated by 10000 tuples, where the first element
# of each tuple contains the tensor with the data, and the second element is an integer)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

num_test = 10000 # 10000
random_numbers2 = np.random.choice(np.arange(10000), size=num_test, replace=False)

# Convert the NumPy range to a Python list
subset_indices2 = list(random_numbers2)

# Create the subset using torch.utils.data.Subset
subset_testset = torch.utils.data.Subset(testset, subset_indices2)

# The loaded sets of data, ready to be inputted into a neural net.
# Each "loader" splits all the data into specified batches.  When iterated, there are
# (num of feature maps / num of batches) items, and each item contains a list of length 2,
# where the first element contains a tensor of (num of batches) elements containing the tensors of data,
# and the second element contains a tensor of (num of batches) elements containing the tensors of labels
batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True, pin_memory=True)
testloader = torch.utils.data.DataLoader(subset_testset, batch_size=batch_size, shuffle=False, pin_memory=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 490kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 12.7MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
num_test3 = 1000 # 10000
random_numbers3 = np.random.choice(np.arange(10000), size=num_test3, replace=False)

# Convert the NumPy range to a Python list
subset_indices3 = list(random_numbers3)

# Create the subset using torch.utils.data.Subset
subset_testset3 = torch.utils.data.Subset(testset, subset_indices3)

batch_size = 64

testloader1000 = torch.utils.data.DataLoader(subset_testset3, batch_size=batch_size, shuffle=False, pin_memory=True)

In [7]:
# Sum scaling function
def sum_scaling(logits):
    logits = torch.exp(logits)
    return logits / logits.sum(dim=1, keepdim=True)+1e-10
#     return logits / logits.sum(dim=1, keepdim=True)

# Custom cross-entropy loss function
def custom_cross_entropy_loss(probabilities, targets):
    log_probabilities = torch.log(probabilities)
    nll_loss = F.nll_loss(log_probabilities, targets)
    return nll_loss

In [8]:
def calculate_accuracy(loader, model, device, printprogress=False):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in loader:
            # Move the input and labels to the same device as the model
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            # Get the predicted class with the highest score
            _, predicted = torch.max(outputs.data, 1)

            # Calculate total and correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if printprogress:
                print(f"{total} test batches processed")

    model.train()  # Set the model back to training mode
    return 100 * correct / total


In [9]:
class Params:
    def __init__(self, red=True, units=1, fixedband=False,
                 totBW=None, noisein=False, noiseout=False, lowtemp=False,
                 weightfreqspacing=None, k_th_AWG=None, k_th_AWG_lowtemp=None,
                 k_th_AWG_lowtemp_opt=None, k_th_w=None, k_th_out=None,
                 k_th_w_lowtemp=None, k_th_out_lowtemp=None,
                 k_th_A2D=None, k_th_A2D_lowtemp=None,
                 digitallike=False, noisefactor=4, optics=False,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=1, unitconverter=1,
                 photodiodearea=1e-6, omegaoptical=2*torch.pi*193.41e12,
                 inttime=1e-7, capval=1e-12, semianalog=True, resconn=False,
                 t=None, del_w=None, w_carrier=None, w_carrier_wei=None):

        self.red = red
        self.units = units
        self.fixedband = fixedband
        self.totBW = totBW
        self.noisein = noisein
        self.noiseout = noiseout
        self.lowtemp = lowtemp
        self.weightfreqspacing = weightfreqspacing
        self.k_th_AWG = k_th_AWG
        self.k_th_AWG_lowtemp = k_th_AWG_lowtemp
        self.k_th_AWG_lowtemp_opt = k_th_AWG_lowtemp_opt
        self.k_th_w = k_th_w
        self.k_th_out = k_th_out
        self.k_th_w_lowtemp = k_th_w_lowtemp
        self.k_th_out_lowtemp = k_th_out_lowtemp
        self.k_th_A2D = k_th_A2D
        self.k_th_A2D_lowtemp = k_th_A2D_lowtemp
        self.digitallike = digitallike
        self.noisefactor = noisefactor
        self.optics = optics
        self.fixedpowerflag = fixedpowerflag
        self.fixedpower = fixedpower
        self.includeinputfft = includeinputfft
        self.includeoutputfft = includeoutputfft
        self.scaletoz = scaletoz
        self.scaletoW = scaletoW
        self.wunits = wunits
        self.unitconverter = unitconverter
        self.photodiodearea = photodiodearea
        self.omegaoptical = omegaoptical
        self.inttime = inttime
        self.capval = capval
        self.semianalog = semianalog
        self.resconn = resconn

        self.t = t
        self.del_w = del_w
        self.w_carrier = w_carrier
        self.w_carrier_wei = w_carrier_wei


In [10]:
class ActivationEncoding(nn.Module):

    def __init__(self, params: Params, inputsize, outputsize):
        super(ActivationEncoding, self).__init__()

        self.params = params
        self.N = inputsize
        self.R = outputsize

        self.del_w = params.del_w
        self.w_carrier = params.w_carrier

        if self.params.red:
            self.n0 = 0
            self.r0 = self.R * (self.N - 1) / 2
            self.highestindex = int(2 * ((self.r0 + self.R) + (self.n0 + self.N) * self.R) + 4)
            self.total = int(2 * (self.highestindex - 1))
        else:
            self.n0 = int(self.N * (self.R - 1) / 2)
            self.r0 = 0
            self.highestindex = int(2 * ((self.r0 + self.R) * self.N + (self.n0 + self.N)) + 4)
            self.total = int(2 * (self.highestindex - 1))

    def forward(self, x):
        device = x.device  # Get the device from the input tensor

        batchsize, numpixels = x.shape

        # Generate the n values (1 to N) for the exponential term, on the same device as input
        n_values = torch.arange(1, numpixels + 1, device=device).unsqueeze(0).unsqueeze(0)  # Shape (1, 1, N)

        # Reshape the time vector to allow broadcasting in the computation, move to the input's device
        t = self.params.t.to(device).unsqueeze(1).unsqueeze(2)  # Shape (n_t, 1, 1)

        # Compute the exponential term with broadcasting, ensuring all computations are on the same device
        if self.params.red:
            exp_term = torch.exp(1j * ((self.n0 + n_values) * self.R * self.del_w + self.w_carrier) * t)  # Shape (n_t, 1, N)
        else:
            exp_term = torch.exp(1j * ((self.n0 + n_values) * self.del_w + self.w_carrier) * t)  # Shape (n_t, 1, N)

        # Expand the input tensor to match the shape of the exponential term
        x_expanded = x.unsqueeze(0)  # Shape (1, batch_size, N)

        # Perform element-wise multiplication and summation over the second dimension (N)
        result = torch.sum(x_expanded * exp_term, dim=2)  # Shape (n_t, batch_size)

        # Transpose the result to get shape (batch_size, n_t), and take the real part
        x = torch.real(result.transpose(0, 1))  # Shape (batch_size, n_t)

        if self.params.includeinputfft:
            x_complex = torch.complex(x, torch.zeros_like(x, device=device))  # Ensure zeros are on the correct device
            x = torch.fft.fft(x_complex.conj().t(), dim=0, norm="ortho").conj().t()

        if self.params.red:
            # inputs_extended = x.unsqueeze(2)
            # extendedshape = inputs_extended.shape
            # zeropaddingleft = torch.zeros(extendedshape[0], extendedshape[1], self.R - 1, device=device)
            # output_inter_tensor = torch.cat([zeropaddingleft, inputs_extended], dim=-1)
            # output_inter_tensor = output_inter_tensor.view(extendedshape[0], -1)
            # furtherzeropaddingleft = torch.zeros(extendedshape[0], self.n0 * self.R + 1, device=device)
            # output_inter_tensor = torch.cat([furtherzeropaddingleft, output_inter_tensor], dim=-1)
            # zeropaddingright = torch.zeros(extendedshape[0], self.highestindex - output_inter_tensor.shape[-1], device=device)
            # y = torch.cat([output_inter_tensor, zeropaddingright], dim=-1)
            # # x = torch.real(torch.fft.irfft(y, dim=-1, norm="forward"))
            # x = torch.real(torch.fft.irfft(y, dim=-1, norm="ortho"))

            if self.params.fixedpowerflag:
                avgpowx = torch.mean(x ** 2, dim=-1, keepdim=True) / 50
                multfact = self.params.fixedpower / avgpowx
                x = x * multfact

            if not self.params.optics:
                if not self.params.fixedband:
                    xener = torch.sum(x ** 2, dim=-1) * (1 / self.params.weightfreqspacing) / self.total
                else:
                    xener = torch.sum(x ** 2, dim=-1) * (1 / self.params.totBW)
            else:
                xener = torch.sum(x ** 2, dim=-1)
        else:
            # xshape = x.shape
            # zeropaddingleft = torch.zeros(xshape[0], self.n0 + 1, device=device)
            # output_inter_tensor = torch.cat([zeropaddingleft, x], dim=-1)
            # zeropaddingright = torch.zeros(xshape[0], self.highestindex - output_inter_tensor.shape[-1], device=device)
            # y = torch.cat([output_inter_tensor, zeropaddingright], dim=-1)
            # # x = torch.real(torch.fft.irfft(y, dim=-1, norm="forward"))
            # x = torch.real(torch.fft.irfft(y, dim=-1, norm="ortho"))

            if self.params.fixedpowerflag:
                avgpowx = torch.mean(x ** 2, dim=-1, keepdim=True) / 50
                multfact = self.params.fixedpower / avgpowx
                x = x * multfact

            if not self.params.optics:
                if not self.params.fixedband:
                    xener = torch.sum(x ** 2, dim=-1) * (1 / self.params.weightfreqspacing) / self.total
                else:
                    xener = torch.sum(x ** 2, dim=-1) * (1 / self.params.totBW)
            else:
                xener = torch.sum(x ** 2, dim=-1)

        # Adding noise in the time domain
        if self.params.noisein:
            if not self.params.optics:
                noisetouse = self.params.k_th_AWG_lowtemp if self.params.lowtemp else self.params.k_th_AWG
                sigma2_x_th = noisetouse * self.params.weightfreqspacing * self.total if not self.params.fixedband else noisetouse * self.params.totBW
                if self.params.digitallike:
                    sigma2_x_th = sigma2_x_th * self.params.noisefactor
                xadditivenoise = torch.normal(0., torch.sqrt(sigma2_x_th), size=x.shape, device=device)
                x = x + xadditivenoise

        return x, xener


In [11]:
class WeightEncodingandMixing(nn.Module):

    def __init__(self, params: Params, inputsize, outputsize, actshape=None):
        super(WeightEncodingandMixing, self).__init__()

        self.params = params
        self.N = inputsize
        self.R = outputsize
        self.actshape = actshape

        self.del_w = params.del_w
        self.w_carrier_wei = params.w_carrier_wei

        if self.params.red:
            self.n0 = 0
            self.r0 = self.R * (self.N - 1) / 2
            self.highestindex = int(2 * ((self.r0 + self.R) + (self.n0 + self.N) * self.R) + 4)
            self.total = int(2 * (self.highestindex - 1))
        else:
            self.n0 = int(self.N * (self.R - 1) / 2)
            self.r0 = 0
            self.highestindex = int(2 * ((self.r0 + self.R) * self.N + (self.n0 + self.N)) + 4)
            self.total = int(2 * (self.highestindex - 1))

        if self.params.semianalog:
            self.W = nn.Parameter(torch.empty(outputsize, inputsize))
            nn.init.kaiming_uniform_(self.W, a=0)
        else:
            self.W = nn.Parameter(torch.empty(outputsize, inputsize))
            # self.W = nn.Parameter(torch.empty(int(1), int(self.actshape[1] / 2 + 1)))
            nn.init.kaiming_uniform_(self.W, a=0)

    def forward(self, z):
        device = z.device  # Get the device from the input tensor

        if self.params.semianalog:
            if self.params.red:
                if not self.params.includeinputfft and not self.params.includeoutputfft:
                    Wtr = self.W.t()
                else:
                    selfW = self.W.to(dtype=torch.complex64)

                    if self.params.includeoutputfft:
                        selfW = torch.fft.fft(selfW, dim=0, norm="ortho")

                    if self.params.includeinputfft:
                        selfW = torch.fft.fft(selfW.conj().t(), dim=0, norm="ortho").conj().t()

                    Wtr = selfW.t()

                Wunrolled = Wtr.reshape(-1)
                zeropaddingleft = torch.zeros(int(self.r0 + 1 + (self.n0 + 1) * self.R), device=device)
                Wpaddedleft = torch.cat([zeropaddingleft, Wunrolled], dim=-1)
                zeropaddingright = torch.zeros(self.highestindex - Wpaddedleft.shape[-1], device=device)
                Wpadded = torch.cat([Wpaddedleft, zeropaddingright], dim=-1)
                # Wtime = torch.real(torch.fft.irfft(Wpadded, dim=-1, norm="forward").unsqueeze(0))
                Wtime = torch.real(torch.fft.irfft(Wpadded, dim=-1, norm="ortho").unsqueeze(0))
            else:
                Wunrolled = self.W.reshape(-1)
                zeropaddingleft = torch.zeros(int((self.r0 + 1) * self.N + (self.n0 + 1)), device=device)
                Wpaddedleft = torch.cat([zeropaddingleft, Wunrolled], dim=-1)
                zeropaddingright = torch.zeros(self.highestindex - Wpaddedleft.shape[-1], device=device)
                Wpadded = torch.cat([Wpaddedleft, zeropaddingright], dim=-1)
                # Wtime = torch.real(torch.fft.irfft(Wpadded, dim=-1, norm="forward").unsqueeze(0))
                Wtime = torch.real(torch.fft.irfft(Wpadded, dim=-1, norm="ortho").unsqueeze(0))

            if self.params.noisein:
                if not self.params.optics:
                    noisetouse = self.params.k_th_w_lowtemp if self.params.lowtemp else self.params.k_th_w
                    sigma2_W_th = noisetouse * self.params.weightfreqspacing * self.total if not self.params.fixedband else noisetouse * self.params.totBW
                    if self.params.digitallike:
                        sigma2_W_th = sigma2_W_th * self.params.noisefactor
                    Wadditivenoise = torch.normal(0., torch.sqrt(sigma2_W_th), size=Wtime.shape, device=device)
                    Wtime = Wtime + Wadditivenoise
        else:

            # Define the time vector, move it to the correct device, and reshape
            t = self.params.t.to(device).unsqueeze(1).unsqueeze(2)  # Shape (n_t, 1, 1)

            # Define the 2D tensor W_rn with shape (R, N), then expand to 3D
            W_rn = self.W.to(device).unsqueeze(0)  # Shape (1, R, N)

            # Generate r and n indices and expand to match dimensions, ensure they are on the correct device
            r_values = torch.arange(1, self.R + 1, device=device).view(1, self.R, 1)  # Shape (1, R, 1)
            n_values = torch.arange(1, self.N + 1, device=device).view(1, 1, self.N)  # Shape (1, 1, N)

            # Compute ω^W_{r,n} as a 3D tensor on the same device
            if self.params.red:
                w_offset_weights = (self.r0 + r_values) * self.del_w + (self.n0 + n_values) * self.R * self.del_w  # Shape (1, R, N)
            else:
                w_offset_weights = (self.r0 + r_values) * self.R * self.del_w + (self.n0 + n_values) * self.del_w

            # Compute the exponential term with broadcasting over time
            exp_term = torch.exp(1j * (w_offset_weights + self.w_carrier_wei) * t)  # Shape (n_t, R, N)

            # Perform element-wise multiplication and summation over r and n dimensions
            result = torch.sum(W_rn * exp_term, dim=(1, 2))  # Shape (n_t,)

            # Unsqueeze the result to get shape (1, n_t), and take the real part
            Wtime = torch.real(result.unsqueeze(0))  # Shape (1, n_t)

            # Wtime = torch.real(torch.fft.irfft(self.W, dim=-1, norm="forward"))
            # Wtime = torch.real(torch.fft.irfft(self.W, dim=-1, norm="ortho"))

        if not self.params.optics:
            # print(f"z device is {z.device}")
            # print(f"Wtime device is {Wtime.device}")

            # small signal multiplication --> amplitudes get killed
            print(f"z shape is {z.shape}")
            print(f"Wtime shape is {Wtime.shape}")
            outputs_in_time = (1/(4*(kb*T/eleccharge))) * torch.mul(z, Wtime)
            # outputs_in_time = torch.mul(torch.sign(z), Wtime)
            # outputs_in_time = torch.mul(z, torch.sign(Wtime))

            # exact formula (non diff?)
            # VT = kb*T/eleccharge
            # outputs_in_time = Wtime/2 + (VT/2)*torch.log((torch.exp(z/VT)+torch.exp(-Wtime/VT))/(torch.exp(z/VT)+torch.exp(Wtime/VT)))
        else:
            upper_out = (z + Wtime) / torch.sqrt(torch.tensor(2, device=device))
            lower_out = (z - Wtime) / torch.sqrt(torch.tensor(2, device=device))
            nth = boseeinstein(self.params.omegaoptical, T)
            upper_var = (nth + nth**2 + (2 * nth + 1) * upper_out**2)
            lower_var = (nth + nth**2 + (2 * nth + 1) * lower_out**2)
            diff_current = eleccharge * torch.normal(mean=2 * z * Wtime, std=torch.sqrt(upper_var + lower_var), device=device)
            outputs_in_time = diff_current / self.params.capval

        if not self.params.optics and self.params.scaletoz:
            outputs_in_time = F.normalize(outputs_in_time, p=2, dim=-1)
            znorm = torch.norm(z, p=2, dim=-1, keepdim=True)
            outputs_in_time = torch.mul(znorm, outputs_in_time)

        if not self.params.optics and self.params.scaletoW:
            outputs_in_time = F.normalize(outputs_in_time, p=2, dim=-1)
            Wnorm = torch.norm(Wtime, p=2, dim=-1, keepdim=True)
            outputs_in_time = torch.mul(Wnorm, outputs_in_time)

        outputs_in_time = outputs_in_time * self.params.unitconverter

        if self.params.noiseout:
            noisetouse = self.params.k_th_out_lowtemp if self.params.lowtemp else self.params.k_th_out
            sigma2_output_th = noisetouse * self.params.weightfreqspacing * self.total if not self.params.fixedband else noisetouse * self.params.totBW
            if not self.params.digitallike:
                output_additivenoise = torch.normal(0., torch.sqrt(sigma2_output_th), size=Wtime.shape, device=device)
                outputs_in_time = outputs_in_time + output_additivenoise

        return outputs_in_time


In [12]:
class A2D(nn.Module):

    def __init__(self, params: Params, inputsize, outputsize):
        super(A2D, self).__init__()

        self.params = params

        # if self.params.noiseout:
        self.N = inputsize
        self.R = outputsize

        if self.params.red:
            self.n0 = 0
            self.r0 = self.R * (self.N - 1) / 2
            self.highestindex = int(2 * ((self.r0 + self.R) + (self.n0 + self.N) * self.R) + 4)
            self.total = int(2 * (self.highestindex - 1))
        else:
            self.n0 = int(self.N * (self.R - 1) / 2)
            self.r0 = 0
            self.highestindex = int(2 * ((self.r0 + self.R) * self.N + (self.n0 + self.N)) + 4)
            self.total = int(2 * (self.highestindex - 1))

    def forward(self, outputs_in_time):
        device = outputs_in_time.device  # Get the device of the input tensor

        if self.params.noiseout:
            noisetouse = self.params.k_th_A2D_lowtemp if self.params.lowtemp else self.params.k_th_A2D

            if not self.params.fixedband:
                sigma2_A2D_th = noisetouse * self.params.weightfreqspacing * self.total
            else:
                sigma2_A2D_th = noisetouse * self.params.totBW

            if self.params.semianalog:
                sigma2_A2D_th = sigma2_A2D_th * self.params.noisefactor

            if not self.params.digitallike:
                additivenoise = torch.normal(0., torch.sqrt(sigma2_A2D_th), size=outputs_in_time.shape, device=device)
                outputs_in_time = outputs_in_time + additivenoise

        if self.params.optics:
            outputs_in_time = outputs_in_time * self.params.capval / (2 * eleccharge)
        else:
            outputs_in_time = outputs_in_time * 4 * (kb*T/eleccharge)

        # Perform the FFT on the same device
        # outputs_in_freq = torch.fft.rfft(outputs_in_time, dim=-1, norm="forward")
        # outputs_in_freq = torch.sqrt(torch.tensor(self.total)) * torch.fft.rfft(outputs_in_time, dim=-1, norm="ortho")

        # to get things to work for the time-domain carrier stuff
        outputs_in_freq = torch.sqrt(torch.tensor(self.total)) * torch.real(torch.fft.rfft(outputs_in_time, dim=-1, norm="ortho"))

        return outputs_in_freq


In [13]:
class MAFTFilter_TimeOutput(nn.Module):

    def __init__(self, params: Params, inputsize, outputsize):
        super(MAFTFilter_TimeOutput, self).__init__()

        self.params = params
        self.N = inputsize
        self.R = outputsize

        if self.params.red:
            self.n0 = 0
            self.r0 = self.R * (self.N - 1) / 2
            self.highestindex = int(2 * ((self.r0 + self.R) + (self.n0 + self.N) * self.R) + 4)
            self.total = int(2 * (self.highestindex - 1))
        else:
            self.n0 = int(self.N * (self.R - 1) / 2)
            self.r0 = 0
            self.highestindex = int(2 * ((self.r0 + self.R) * self.N + (self.n0 + self.N)) + 4)
            self.total = int(2 * (self.highestindex - 1))

    def forward(self, z):
        device = z.device  # Get the device of the input tensor

        if self.params.red:
            z[:, :int(self.r0 + 1)] = 0
            z[:, int(self.r0 + self.R + 1):] = 0
            outputs_in_time = torch.fft.irfft(z, dim=-1, norm="ortho") / torch.sqrt(torch.tensor(self.total))
            # outputs_in_freq = torch.sqrt(torch.tensor(self.total)) * torch.fft.rfft(outputs_in_time, dim=-1, norm="ortho")
            # z = z[:, int(self.r0 + 1):int(self.r0 + self.R + 1)]
        # else:
        #     z = z[:, int((self.r0 + 1) * self.N):int((self.r0 + self.R) * self.N + 1):int(self.N)]

        # if self.params.includeoutputfft:
        #     z = torch.fft.ifft(z.t(), dim=0, norm="ortho").t().real

        if self.params.optics:
            outputs_in_time = outputs_in_time / (self.params.capval / (2 * eleccharge))
        else:
            outputs_in_time = outputs_in_time / (4 * (kb*T/eleccharge))

        return outputs_in_time.real


In [14]:
class MAFTFilter(nn.Module):

    def __init__(self, params: Params, inputsize, outputsize):
        super(MAFTFilter, self).__init__()

        self.params = params
        self.N = inputsize
        self.R = outputsize

        if self.params.red:
            self.r0 = self.R * (self.N - 1) / 2
        else:
            self.r0 = 0

    def forward(self, z):
        device = z.device  # Get the device of the input tensor

        if self.params.red:
            z = z[:, int(self.r0 + 1):int(self.r0 + self.R + 1)]
        else:
            z = z[:, int((self.r0 + 1) * self.N):int((self.r0 + self.R) * self.N + 1):int(self.N)]

        if self.params.includeoutputfft:
            z = torch.fft.ifft(z.t(), dim=0, norm="ortho").t().real

        return z.real


In [15]:
class SquareNonlinorig(nn.Module):

    def __init__(self):

        super(SquareNonlinorig, self).__init__()


    def forward(self, z):

        z = torch.square(z)
        z = z/torch.max(z, dim=1, keepdim=True).values

        return z


In [16]:
class SquareNonlin(nn.Module):

    def __init__(self):

        super(SquareNonlin, self).__init__()


    def forward(self, z):

        z = (1/(4*(kb*T/eleccharge))) * torch.square(z)/2

        return z

In [17]:
class MyNonlin(nn.Module):

    def __init__(self):

        super(MyNonlin, self).__init__()


    def forward(self, z):

        z = torch.nn.functional.relu(z)

        return z

In [18]:
class ExactNonlin(nn.Module):

    def __init__(self):

        super(ExactNonlin, self).__init__()


    def forward(self, z):

        VT = kb*T/eleccharge
        z = z/2 + (VT/2)*torch.log((torch.exp(z/VT)+torch.exp(-z/VT))/(torch.exp(z/VT)+torch.exp(z/VT)))

        return z

In [19]:
class AnaNetwork(nn.Module):

    def __init__(self, params: Params, analogL, numpixels, analognumhiddenunits, numclasses):
        super(AnaNetwork, self).__init__()

        self.params = params
        self.analogL = analogL
        self.activations = []
        self.alternate = False  # You can modify this if needed
        self.layers = nn.ModuleDict()

        self.layers["flatten"] = nn.Flatten()

        for i in np.arange(analogL + 1):
            if i == 0:
                inputsize = numpixels
                outputsize = analognumhiddenunits[i]
            elif i == analogL:
                inputsize = analognumhiddenunits[i - 1]
                outputsize = numclasses
            else:
                inputsize = analognumhiddenunits[i - 1]
                outputsize = analognumhiddenunits[i]

            red = True if (not self.alternate or i % 2 == 0) else False

            if self.params.semianalog:
                # Analog pre-processing layer
                self.layers[f"analogprepro{i+1}"] = ActivationEncoding(self.params, inputsize, outputsize)

                # Analog weight encoding and mixing layer
                self.layers[f"analoglayer{i+1}"] = WeightEncodingandMixing(self.params, inputsize, outputsize)

                # A2D conversion layer
                self.layers[f"A2D{i+1}"] = A2D(self.params, inputsize, outputsize)

                # MAFT filter layer
                self.layers[f"filter{i+1}"] = MAFTFilter(self.params, inputsize, outputsize)

                # Non-linearity layer
                self.layers[f"analognonlin{i+1}"] = MyNonlin()

            else:
                if i == 0:
                    # Analog pre-processing layer with no noise
                    self.layers[f"analogprepro{i+1}"] = ActivationEncoding(self.params, inputsize, outputsize)
                    totaltuple = (1, self.layers[f"analogprepro{i+1}"].total)

                # Analog weight encoding and mixing layer
                self.layers[f"analoglayer{i+1}"] = WeightEncodingandMixing(self.params, inputsize, outputsize,
                                                                           actshape=totaltuple)

                # Dropout layer
                # self.layers[f"dropout{i+1}"] = nn.Dropout(0.3)

                # conv layer
                self.layers[f"conv1d{i+1}"] = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding='same')

                # A2D conversion layer
                self.layers[f"A2D{i+1}"] = A2D(self.params, inputsize, outputsize)

                # MAFT filter timeoutput layer
                self.layers[f"filtertime{i+1}"] = MAFTFilter_TimeOutput(self.params, inputsize, outputsize)

                # MAFT filter layer
                self.layers[f"filter{i+1}"] = MAFTFilter(self.params, inputsize, outputsize)

                # Non-linearity layer
                self.layers[f"analognonlin{i+1}"] = MyNonlin()
                # self.layers[f"analognonlin{i+1}"] = SquareNonlin()
                # self.layers[f"analognonlin{i+1}"] = ExactNonlin()

            if self.params.resconn:
                self.layers[f"analogclip{i+1}"] = MAFTClip(outputsize)
                self.layers[f"analogavg{i+1}"] = MAFTAvg(inputsize, outputsize)

        # Register hooks to save activations
        for name, layer in self.layers.items():
            if 'analogprepro' in name:
                layer.register_forward_hook(self.save_activation)

    def save_activation(self, module, input, output):
        device = output[1].device  # Ensure that activations are tracked on the correct device
        _, output2 = output
        self.activations.append(output2.to(device))  # Move activations to the correct device if necessary

    def get_sum_of_squares(self):
        sum_of_squares = 0
        for activation in self.activations:
            sum_of_squares += torch.sum(activation)
        return sum_of_squares

    def forward(self, x):
        self.activations = []  # Clear previous activations
        device = x.device  # Extract device from input tensor
        x = self.layers["flatten"](x)

        for i in np.arange(self.analogL + 1):
            if self.params.resconn:
                if not self.params.avg:
                    xclip = self.layers[f"analogclip{i+1}"](x.to(device))  # Ensure xclip is on the correct device
                else:
                    xclip = self.layers[f"analogavg{i+1}"](x.to(device))

            if self.params.semianalog:
                x, _ = self.layers[f"analogprepro{i+1}"](x.to(device))
                x = self.layers[f"analoglayer{i+1}"](x)
                x = self.layers[f"A2D{i+1}"](x)
                x = self.layers[f"filter{i+1}"](x)

                if i != self.analogL:
                    x = self.layers[f"analognonlin{i+1}"](x)

                if self.params.resconn:
                    x = x + xclip  # Residual connection on the correct device

            else:
                if i == 0:
                    x, _ = self.layers[f"analogprepro{i+1}"](x.to(device))
                    # print(f"Energy after actencoding is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                x = self.layers[f"analoglayer{i+1}"](x)
                # print(f"Energy after mixer is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")
                x = self.layers[f"A2D{i+1}"](x)

                x = self.layers[f"filtertime{i+1}"](x)
                # x = self.layers[f"filter{i+1}"](x) # only for 1 layer

                # print energy in x
                # print(f"Energy after filter is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                # print(x.shape)
                # x = (self.layers[f"conv1d{i+1}"](x.unsqueeze(1))).squeeze()
                # print(x.shape)
                # x = self.layers[f"dropout{i+1}"](x)

                if i != self.analogL:
                    x = self.layers[f"analognonlin{i+1}"](x)
                    # print energy in x
                    # print(f"Energy after nonlinearity is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                if i == self.analogL:
                    x = self.layers[f"A2D{i+1}"](x)
                    x = self.layers[f"filter{i+1}"](x)

                if self.params.resconn:
                    x = x + xclip  # Residual connection on the correct device

        return x


In [20]:
class AnaNetwork1layer(nn.Module):

    def __init__(self, params: Params, analogL, numpixels, numclasses):
        super(AnaNetwork1layer, self).__init__()

        self.params = params
        self.analogL = analogL
        self.activations = []
        self.alternate = False  # You can modify this if needed
        self.layers = nn.ModuleDict()

        self.layers["flatten"] = nn.Flatten()

        for i in np.arange(analogL + 1):
            if analogL==0:
                inputsize = numpixels
                outputsize = numclasses
            else:
                if i == 0:
                    inputsize = numpixels
                    outputsize = analognumhiddenunits[i]
                elif i == analogL:
                    inputsize = analognumhiddenunits[i - 1]
                    outputsize = numclasses
                else:
                    inputsize = analognumhiddenunits[i - 1]
                    outputsize = analognumhiddenunits[i]

            red = True if (not self.alternate or i % 2 == 0) else False

            if i == 0:
                # Analog pre-processing layer with no noise
                self.layers[f"analogprepro{i+1}"] = ActivationEncoding(self.params, inputsize, outputsize)
                totaltuple = (1, self.layers[f"analogprepro{i+1}"].total)

            # Analog weight encoding and mixing layer
            self.layers[f"analoglayer{i+1}"] = WeightEncodingandMixing(self.params, inputsize, outputsize,
                                                                        actshape=totaltuple)

            # Dropout layer
            # self.layers[f"dropout{i+1}"] = nn.Dropout(0.3)

            # conv layer
            # self.layers[f"conv1d{i+1}"] = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding='same')

            # A2D conversion layer
            self.layers[f"A2D{i+1}"] = A2D(self.params, inputsize, outputsize)

            # MAFT filter timeoutput layer
            self.layers[f"filtertime{i+1}"] = MAFTFilter_TimeOutput(self.params, inputsize, outputsize)

            # MAFT filter layer
            self.layers[f"filter{i+1}"] = MAFTFilter(self.params, inputsize, outputsize)

            # Non-linearity layer
            self.layers[f"analognonlin{i+1}"] = MyNonlin()
            # self.layers[f"analognonlin{i+1}"] = SquareNonlin()
            # self.layers[f"analognonlin{i+1}"] = ExactNonlin()

        # Register hooks to save activations
        for name, layer in self.layers.items():
            if 'analogprepro' in name:
                layer.register_forward_hook(self.save_activation)

    def save_activation(self, module, input, output):
        device = output[1].device  # Ensure that activations are tracked on the correct device
        _, output2 = output
        self.activations.append(output2.to(device))  # Move activations to the correct device if necessary

    def get_sum_of_squares(self):
        sum_of_squares = 0
        for activation in self.activations:
            sum_of_squares += torch.sum(activation)
        return sum_of_squares

    def forward(self, x):
        self.activations = []  # Clear previous activations
        device = x.device  # Extract device from input tensor
        x = self.layers["flatten"](x)

        for i in np.arange(self.analogL + 1):
            if self.params.resconn:
                if not self.params.avg:
                    xclip = self.layers[f"analogclip{i+1}"](x.to(device))  # Ensure xclip is on the correct device
                else:
                    xclip = self.layers[f"analogavg{i+1}"](x.to(device))

            if self.params.semianalog:
                x, _ = self.layers[f"analogprepro{i+1}"](x.to(device))
                x = self.layers[f"analoglayer{i+1}"](x)
                x = self.layers[f"A2D{i+1}"](x)
                x = self.layers[f"filter{i+1}"](x)

                if i != self.analogL:
                    x = self.layers[f"analognonlin{i+1}"](x)

                if self.params.resconn:
                    x = x + xclip  # Residual connection on the correct device

            else:
                if i == 0:
                    x, _ = self.layers[f"analogprepro{i+1}"](x.to(device))
                    print(f"Activations after analogprepro {x[0]}")
                    # print(f"Energy after actencoding is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                x = self.layers[f"analoglayer{i+1}"](x)
                print(f"Activations after analoglayer {x[0]}")
                # print(f"Energy after mixer is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")
                x = self.layers[f"A2D{i+1}"](x)
                print(f"Activations after A2D {x[0]}")
                x = self.layers[f"filter{i+1}"](x)
                print(f"Activations after filter {x[0]}")

                # print energy in x
                # print(f"Energy after filter is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                # print(x.shape)
                # x = (self.layers[f"conv1d{i+1}"](x.unsqueeze(1))).squeeze()
                # print(x.shape)
                # x = self.layers[f"dropout{i+1}"](x)

                # if i != self.analogL:
                    # x = self.layers[f"analognonlin{i+1}"](x)
                    # print energy in x
                    # print(f"Energy after nonlinearity is {torch.sum(x[0] ** 2) * (1 / self.params.totBW)/50}")

                if i == self.analogL:
                    x = self.layers[f"A2D{i+1}"](x)
                    x = self.layers[f"filter{i+1}"](x)

                if self.params.resconn:
                    x = x + xclip  # Residual connection on the correct device

        return x


In [21]:
numpixels = 7*7
analogL = 2
analognumhiddenunits = [32, 16]
numclasses = 10

In [22]:
noisein = True
noiseout = True

lowtemp = True

noisefactor = torch.tensor(4)

preprounits = torch.tensor(1) # current in A
wunits = torch.tensor(1) # voltage in V
unitconverter = torch.tensor(1) # converts voltage to conductance,
                                # and also has a unitless voltage divider like factor

kb = 1.38e-23
T = 300
hbar = 1.05e-34

optics = False
red = True
omegac = 2*torch.pi*1e9 # let's say 5 GHz is the center frequency

weightfreqspacing = torch.tensor(1e3) # in Hz, 1 kHz
timespan = 1/weightfreqspacing
totBW = torch.tensor(1e8) # in Hz, 100 MHz
numcomblines = totBW/weightfreqspacing # which is the number of MACs

R_trans = torch.tensor(50) # ohms
RF_power = torch.tensor(250e-6) # Watts
Vref = torch.sqrt(RF_power*R_trans)

analogenergyperMAC = RF_power*timespan/numcomblines

In [25]:
R_AWG = 50
R_w = 50
R_out = 50
R_A2D = 50

k_th_AWG = 4*kb*T*R_AWG
k_th_w = 4*kb*T*R_w
k_th_out = 4*kb*T*R_out
k_th_A2D = 4*kb*T*R_A2D

k_th_AWG_lowtemp = 2*(hbar*omegac*np.cosh(hbar*omegac/(2*kb*T))/np.sinh(hbar*omegac/(2*kb*T)))*R_AWG
k_th_w_lowtemp = 2*(hbar*omegac*np.cosh(hbar*omegac/(2*kb*T))/np.sinh(hbar*omegac/(2*kb*T)))*R_w
k_th_out_lowtemp = 2*(hbar*omegac*np.cosh(hbar*omegac/(2*kb*T))/np.sinh(hbar*omegac/(2*kb*T)))*R_out
k_th_A2D_lowtemp = 2*(hbar*omegac*np.cosh(hbar*omegac/(2*kb*T))/np.sinh(hbar*omegac/(2*kb*T)))*R_A2D

In [26]:
paramsfullyanalog.device

NameError: name 'paramsfullyanalog' is not defined

In [23]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

numpixels = 7*7
# numpixels = 14*14
# numpixels = 28*28

analogL = 0

# analognumhiddenunits = [32, 16]
# analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

N = numpixels
R = numclasses
n0 = 0
r0 = R * (N - 1) / 2
highestindex = int(2 * ((r0 + R) + (n0 + N) * R) + 4)

tmax = 1/(totBW/highestindex)
tstep = 1/totBW
t = torch.arange(0, tmax+tstep, tstep)

del_w = (totBW/highestindex)
w_carrier = torch.tensor(2*np.pi*0.4e9)
w_carrier_wei = torch.tensor(2*np.pi*0.8e9)

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False,
                           t=t, del_w=del_w, w_carrier=w_carrier, w_carrier_wei=w_carrier_wei)

for modelno in range(6, 7):

    # model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    model = AnaNetwork1layer(paramsfullyanalog, analogL, numpixels, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-14
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            # outputs = model(scalefactlocal*images)
            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

NameError: name 'k_th_AWG' is not defined

In [ ]:
model = AnaNetwork1layer(paramsfullyanalog, analogL, numpixels, numclasses).to(device)

for images, labels in trainloader:
    print(model(images[0]))
    break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

numpixels = 7*7
# numpixels = 14*14
# numpixels = 28*28

analogL = 2

analognumhiddenunits = [32, 16]
# analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-14
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            # outputs = model(scalefactlocal*images)
            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.1857,        Training Accuracy: 61.39%, Validation Accuracy: 61.30%,        Time: 36.92517280578613
Epoch [2/100], Loss: 0.8598,        Training Accuracy: 69.78%, Validation Accuracy: 69.27%,        Time: 35.405282855033875
Epoch [3/100], Loss: 0.8099,        Training Accuracy: 74.43%, Validation Accuracy: 74.64%,        Time: 35.23499393463135
Epoch [4/100], Loss: 1.0763,        Training Accuracy: 78.52%, Validation Accuracy: 78.64%,        Time: 34.78070646524429
Epoch [5/100], Loss: 0.6213,        Training Accuracy: 81.33%, Validation Accuracy: 81.43%,        Time: 34.68626856803894
Epoch [6/100], Loss: 0.4542,        Training Accuracy: 83.80%, Validation Accuracy: 83.66%,        Time: 34.63933348655701
Epoch [7/100], Loss: 0.4990,        Training Accuracy: 84.86%, Validation Accuracy: 84.69%,        Time: 34.44905877113342


KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
# numpixels = 14*14
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-14
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            outputs = model(scalefactlocal*images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 2.3026,        Training Accuracy: 9.61%, Validation Accuracy: 10.36%,        Time: 451.5763108730316


KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
# numpixels = 14*14
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-14
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            outputs = model(scalefactlocal*images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 2.3026,        Training Accuracy: 10.26%, Validation Accuracy: 10.00%,        Time: 505.6519498825073


KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
# numpixels = 14*14
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-18
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            outputs = model(scalefactlocal*images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 2.3026,        Training Accuracy: 8.25%, Validation Accuracy: 7.98%,        Time: 404.31306409835815


KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

numpixels = 7*7
# numpixels = 14*14
# numpixels = 28*28

analogL = 2

analognumhiddenunits = [32, 16]
# analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    energypermaclocal = 1e-18
    R_trans = 50
    delta_t = 1/paramsfullyanalog.totBW

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")

            scalefactlocal = torch.sqrt(energypermaclocal * R_trans / (delta_t))
            outputs = model(scalefactlocal*images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Energy after actencoding is 1.4562314307764467e-15
Energy after mixer is 1.6052859435317796e-18
Energy after filter is 7.472750865494402e-22
Energy after nonlinearity is 6.626958367729232e-37
Energy after mixer is 8.134649689282709e-40
Energy after filter is 1.471363387541058e-43
Energy after nonlinearity is 0.0
Energy after mixer is 0.0
Energy after filter is 0.0
Energy after actencoding is 1.411501950172552e-15
Energy after mixer is 1.6045778769274342e-18
Energy after filter is 5.739263736151059e-22
Energy after nonlinearity is 4.801053464664246e-37
Energy after mixer is 5.888031939338589e-40
Energy after filter is 1.401298464324817e-43
Energy after nonlinearity is 0.0
Energy after mixer is 0.0
Energy after filter is 0.0
Energy after actencoding is 1.4070480391776522e-15
Energy after mixer is 1.5706326593349425e-18
Energy after filter is 5.310427336040392e-22
Energy after nonlinearity is 2.9341604694138825e-37
Energy after mixer is 3.613304142200114e-40
Energy after filter is 1.03696

KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
# numpixels = 14*14
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [64, 32]
# analognumhiddenunits = [100, 100]
# analognumhiddenunits = [50, 50]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = False

noiseout = False

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=False, includeoutputfft=False,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")
            outputs = model(10*images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.0889,        Training Accuracy: 64.68%, Validation Accuracy: 65.69%,        Time: 397.21464800834656
Epoch [2/100], Loss: 0.2454,        Training Accuracy: 83.28%, Validation Accuracy: 83.56%,        Time: 397.4351716041565
Epoch [3/100], Loss: 0.9671,        Training Accuracy: 87.02%, Validation Accuracy: 87.33%,        Time: 397.45677439371747
Epoch [4/100], Loss: 0.0739,        Training Accuracy: 88.17%, Validation Accuracy: 88.10%,        Time: 397.60210359096527
Epoch [5/100], Loss: 0.1795,        Training Accuracy: 89.39%, Validation Accuracy: 89.40%,        Time: 397.6714231967926
Epoch [6/100], Loss: 0.1092,        Training Accuracy: 90.84%, Validation Accuracy: 91.13%,        Time: 397.72291747728985
Epoch [7/100], Loss: 0.1690,        Training Accuracy: 91.60%, Validation Accuracy: 91.71%,        Time: 397.775438444955
Epoch [8/100], Loss: 0.3167,        Training Accuracy: 92.15%, Validation Accuracy: 92.28%,        Time: 397.7806761562824
Epoch [9/100]

KeyboardInterrupt: 

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [100, 100]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = True

noiseout = True

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=True, includeoutputfft=True,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 100
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")
            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.7079,        Training Accuracy: 56.25%, Validation Accuracy: 56.14%,        Time: 179.96079754829407
Epoch [2/100], Loss: 1.2817,        Training Accuracy: 61.55%, Validation Accuracy: 61.38%,        Time: 179.14620971679688
Epoch [3/100], Loss: 1.5885,        Training Accuracy: 64.10%, Validation Accuracy: 64.09%,        Time: 178.93843817710876
Epoch [4/100], Loss: 1.2135,        Training Accuracy: 66.20%, Validation Accuracy: 66.09%,        Time: 178.8510321378708
Epoch [5/100], Loss: 1.6259,        Training Accuracy: 67.20%, Validation Accuracy: 67.06%,        Time: 178.75732870101928
Epoch [6/100], Loss: 0.9910,        Training Accuracy: 68.14%, Validation Accuracy: 67.89%,        Time: 178.63926756381989
Epoch [7/100], Loss: 0.8932,        Training Accuracy: 67.47%, Validation Accuracy: 67.13%,        Time: 178.6349607535771
Epoch [8/100], Loss: 0.9440,        Training Accuracy: 70.94%, Validation Accuracy: 70.45%,        Time: 178.59112191200256
Epoch [9/1

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

# numpixels = 7*7
numpixels = 28*28

analogL = 2

# analognumhiddenunits = [32, 16]
analognumhiddenunits = [100, 100]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = True

noiseout = True

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=True,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=True, includeoutputfft=True,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 20
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")
            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 1.3796,        Training Accuracy: 58.43%, Validation Accuracy: 58.50%,        Time: 173.70931029319763
Epoch [2/20], Loss: 1.3173,        Training Accuracy: 63.90%, Validation Accuracy: 63.81%,        Time: 172.87598156929016
Epoch [3/20], Loss: 1.2174,        Training Accuracy: 66.23%, Validation Accuracy: 65.58%,        Time: 172.73610790570578
Epoch [4/20], Loss: 0.6843,        Training Accuracy: 67.48%, Validation Accuracy: 67.02%,        Time: 172.6817905306816
Epoch [5/20], Loss: 0.8731,        Training Accuracy: 68.90%, Validation Accuracy: 69.05%,        Time: 172.65811524391174
Epoch [6/20], Loss: 1.4568,        Training Accuracy: 67.61%, Validation Accuracy: 66.85%,        Time: 172.65196216106415
Epoch [7/20], Loss: 1.3731,        Training Accuracy: 72.10%, Validation Accuracy: 71.81%,        Time: 172.6358506679535
Epoch [8/20], Loss: 0.5707,        Training Accuracy: 70.84%, Validation Accuracy: 70.06%,        Time: 172.64169937372208
Epoch [9/20], Loss

In [ ]:
# Initialize the model

# Looping over model training

# inputsize = 784
# hiddenunits = [100, 100]
# outputsize = 10

# inputsize = 49
# hiddenunits = [16, 16]
# outputsize = 10

# inputsize = 196
# hiddenunits = [64, 32]
# outputsize = 10

numpixels = 7*7

analogL = 2

analognumhiddenunits = [32, 16]

# numpixels = 10*10

# analogL = 2

# analognumhiddenunits = [64, 32]

numclasses = 10

noisein = True

noiseout = True

paramsfullyanalog = Params(red=red, units=preprounits, fixedband=False,
                 totBW=totBW, noisein=noisein, noiseout=noiseout, lowtemp=lowtemp,
                 weightfreqspacing=weightfreqspacing, k_th_AWG=k_th_AWG, k_th_AWG_lowtemp=k_th_AWG_lowtemp,
                 k_th_w=k_th_w, k_th_out=k_th_out,
                 k_th_w_lowtemp=k_th_w_lowtemp, k_th_out_lowtemp=k_th_out_lowtemp,
                 k_th_A2D=k_th_A2D, k_th_A2D_lowtemp=k_th_A2D_lowtemp,
                 digitallike=False, noisefactor=noisefactor, optics=optics,
                 fixedpowerflag=False, fixedpower=None,
                 includeinputfft=True, includeoutputfft=True,
                 scaletoz=False, scaletoW=False, wunits=wunits, unitconverter=unitconverter,
                 photodiodearea=1e-6, omegaoptical=omegac,
                 inttime=1e-7, capval=1e-12, semianalog=False, resconn=False)

for modelno in range(6, 7):

    model = AnaNetwork(paramsfullyanalog, analogL, numpixels, analognumhiddenunits, numclasses).to(device)

    num_epochs = 20
    learning_rate = 1e-3

    criterion = nn.CrossEntropyLoss()
#     criterion = custom_cross_entropy_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    start = time.time()

    for epoch in range(num_epochs):

        model.train()

        for i, (images, labels) in enumerate(trainloader):

            # Forward pass
            images, labels = images.to(device), labels.to(device)
            # print(f"batch number is {i}")
            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Calculate training and validation accuracy
        # print(f"epoch {epoch} done")
        train_accuracy = calculate_accuracy(trainloader, model, device)
        # print(f"train acc computed")
        val_accuracy = calculate_accuracy(validloader, model, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
        Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
        Time: {(time.time()-start)/(epoch+1)}')

#     torch.save(model.state_dict(),
#                f"DigiModel/{inputsize}_{hiddenunits[0]}_{hiddenunits[1]}_{outputsize}_{num_epochs}epo/run{modelno}.pth")

#     print(f"Saved model number {modelno}")

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.6674,        Training Accuracy: 73.17%, Validation Accuracy: 72.78%,        Time: 32.64045715332031
Epoch [2/20], Loss: 0.8640,        Training Accuracy: 77.14%, Validation Accuracy: 77.37%,        Time: 31.538044452667236
Epoch [3/20], Loss: 0.3863,        Training Accuracy: 76.74%, Validation Accuracy: 76.59%,        Time: 31.66091235478719
Epoch [4/20], Loss: 0.9778,        Training Accuracy: 73.14%, Validation Accuracy: 73.44%,        Time: 31.344755828380585
Epoch [5/20], Loss: 1.0761,        Training Accuracy: 78.97%, Validation Accuracy: 79.40%,        Time: 31.312420463562013
Epoch [6/20], Loss: 0.4027,        Training Accuracy: 75.48%, Validation Accuracy: 75.08%,        Time: 31.181117177009583
Epoch [7/20], Loss: 0.1805,        Training Accuracy: 76.18%, Validation Accuracy: 76.79%,        Time: 31.117038556507655
Epoch [8/20], Loss: 0.7731,        Training Accuracy: 79.66%, Validation Accuracy: 79.90%,        Time: 31.01620301604271
Epoch [9/20], Loss:

In [ ]:
print(next(model.parameters()).is_cuda)

True


In [ ]:
!nvidia-smi

Wed Oct  2 17:14:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--